# Convert friction surfaces to access surfaces

This notebook generates access-time rasters for each combination of season-mode friction surfaces and destinations

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'

import common_rasterio_ops as rast_ops

import numpy as np
import re

import rasterio
from rasterio import features, transform
from rasterio.merge import merge as merge
from rasterio.transform import Affine

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, MultiPoint, Point, Polygon

import skimage.graph as graph

import json

## Setup

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters 
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_lc_folder = data_loaded['local_lc_folder']
local_boundaries_folder = data_loaded['local_boundaries_folder']
local_destination_folder = data_loaded['local_destination_folder']
fric_dir = data_loaded['fric_dir']
access_dir = data_loaded['access_dir']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']
buffer_m = data_loaded['buffer_m']
level = data_loaded['level']
if level != 'custom':
    shapefile_adm_field = data_loaded['shapefile_adm_field']
    adm_name = data_loaded['adm_name']


seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".tif")])

for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".tif","")   

Destination files

In [ ]:
# Use destinations - assumes they are already projected to the project's metric CRS
dest_fils = {
    re.findall(r'(.*?).gpkg',fil)[0]: gpd.read_file(os.path.join(local_destination_folder,fil))\
    for fil in os.listdir(local_destination_folder) if fil.endswith(".gpkg")
}

Slight adjustment to GOST's TT code: export rasters as Float32 to reduce file sizes

In [ ]:
def get_mcp_dests(inH, destinations, makeset=True):
    ''' Get indices from inH for use in mcp.find_costs
    INPUT
        inH[rasterio] - object from which to extract geographic coordinates
        destinations[geopandas geodataframe] - point geodataframe of destinations
    RETURN
        [list of indices]
    '''
    if makeset:
        cities = list(set([inH.index(x.x, x.y) for x in destinations['geometry']]))
    else:
        cities = list([inH.index(x.x, x.y) for x in destinations['geometry']])
        
    cities = [x for x in cities if ((x[0] > 0) and (x[1] > 0) and 
                (x[0] <= inH.shape[0]) and (x[1] <= inH.shape[1]))]
    return(cities)

In [ ]:
# adjust GOST's code to always export float32 rasters, as a space saving measure

def calculate_travel_time_slim(inH, mcp, destinations, out_raster = ''):
    ''' Calculate travel time raster
    
    INPUTS
        inH [rasterio object] - template raster used to identify locations of destinations
        mcp [skimage.graph.MCP_Geometric] - input graph
        destinations [geopandas df] - destinations for nearest calculations
        
    LINKS
        https://scikit-image.org/docs/0.7.0/api/skimage.graph.mcp.html#skimage.graph.mcp.MCP.find_costs
    '''
    # create skimage graph, force costs to float32 to save on space
    cities = get_mcp_dests(inH, destinations)
    # print(cities)
    costs, traceback = mcp.find_costs(cities)
    # print("costs done")
    costs = costs.astype(np.float32)
    if not out_raster == '':
        meta = inH.meta.copy()
        meta.update(dtype=costs.dtype)
        with rasterio.open(out_raster, 'w', **meta) as out:
            out.write_band(1, costs)
            
    # return((costs, traceback))

## Generate access surfaces

Loop over all friction surfaces, for all destinations, to generate access surfaces for each friction surface + each destination using cost distance methods. Note this may take a while!

In [ ]:
fric_tifs = sorted([os.path.join(fric_dir,file) \
            for file \
            in os.listdir(os.path.join(fric_dir)) \
            if file.endswith(".tif")]) # can filter more aggressively here if useful

for fric_idx, fric in enumerate(fric_tifs):
    
    fric_name = os.path.basename(fric)
    inR = rasterio.open(fric)
    inD = inR.read()[0,:,:] 
    inD = np.nan_to_num(inD,nan=10,posinf=10).astype(np.float32)
    mcp = graph.MCP_Geometric(inD)
    
    for idx, (key, gdf) in enumerate(dest_fils.items()):
        # define labels
        season = re.split(r'_',fric_name)[0]
        modality = re.split(r'_',fric_name)[1].replace('.tif',"")
             
        # print(side)
        
        # countdown
        print(f'{fric_name}: {idx + 1} of {len(dest_fils)}, {key}')

        # # create travel time raster describing travel time from all cells to destinations
        # costs, traceback = 
        calculate_travel_time_slim(inR, mcp, gdf, \
                                                os.path.join(access_dir,f"{season}_{modality}_{key}.tif"))
               
    del inR, inD, mcp